In [226]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pickle as pkl
from os.path import join as oj

import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random

random.seed(10)

from imodels import GreedyRuleListClassifier
from rulevetting.api import validation
from rulevetting.projects.tbi_pecarn.dataset import Dataset
from rulevetting.projects.tbi_pecarn.baseline import Baseline
import rulevetting.projects.tbi_pecarn.helper as helper

MODELS_DIR = './models'
os.makedirs(MODELS_DIR, exist_ok=True)

outcome_def = 'outcome'  # output

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Baseline Model

We need to use the same predictors that appeared in the models in the paper. Our current feature selection methods do not give us the same variables, so I will have to retrieve them. The following predictors are used:

* **AMS**: Altered mental status 
* **Hema**: Scalp haematoma 
* **LocLen**: Loss of conciousness 
* **High_impact_InjSev**: Mech. of injury 
* **SFxPalp**: Palp. or unclear skull fracture 
* **ActNorm**: Acting normally per parent 
* **Vomit**: History of vomiting
* **SFxBas**: Signs of basilar skull fracture 
* **HASeverity**: Severe headache 


We will first load in the raw data set and create a list of the above variable names.

In [228]:
train, tune, test = Dataset().get_data(load_csvs = False)
train = pd.concat([train, tune])
#test = pd.concat([test, tune])
print(train.shape)
print(test.shape)

base_pred = np.array(['AMS_1', 
                     'HemaLoc_2_or_3',
                     'LocLen_2_3_4',
                     'High_impact_InjSev_3', 
                     'SFxPalp_1_or_2', 
                     'ActNorm_0', 
                     'Vomit_1', 
                     'SFxBas_1', 
                     'HASeverity_3', 
                     'AgeTwoPlus_1', # True when under 2 y.o.
                     'outcome'])

young_predrs = np.array(['AMS_1', 
                     'HemaLoc_2_or_3',
                     'LocLen_2_3_4',
                     'High_impact_InjSev_3', 
                     'SFxPalp_1_or_2', 
                     'ActNorm_0', 
                     'outcome'])

old_predrs = np.array(['AMS_1',                      
                     'LocLen_2_3_4',
                     'High_impact_InjSev_3',  
                     'Vomit_1', 
                     'SFxBas_1', 
                     'HASeverity_3', 
                     'outcome'])

train.isna().sum()

(32908, 31)
(8227, 31)


High_impact_InjSev_3    0
Clav_0                  0
VomitNbr_1              0
HemaLoc_1               0
SFxBas_1                0
NeuroD_1                0
AMS_1                   0
FontBulg_1              0
GCSEye_3                0
Seiz_1                  0
HA_verb_1               0
GCSMotor_6              0
VomitStart_2            0
Drugs_1                 0
SFxPalp_1_or_2          0
LOCSeparate_1_or_2      0
SeizOccur_2_or_3        0
SeizLen_3_or_4          0
VomitStart_3_or_4       0
HemaLoc_2_or_3          0
outcome                 0
AMS_1                   0
AgeTwoPlus_1            0
HemaLoc_2_or_3          0
LocLen_2_3_4            0
High_impact_InjSev_3    0
SFxPalp_1_or_2          0
ActNorm_0               0
Vomit_1                 0
SFxBas_1                0
HASeverity_3            0
dtype: int64

## Processing the data

Before we can start the analysis, we will try to process the data in a manner similar to that in the paper. According to Figure 1 in the paper, 42,412 patients remain after accounting for missing outcome values and GCS scores less than 14.

Before further cleaning of the data, we will select the columns that we need for this model.

In [174]:
filt_train = train[base_pred]
filt_test = test[base_pred]
print(filt_train.shape)
print(filt_test.shape)

(32908, 16)
(8227, 16)


The authors of the paper do not remove any entries containing NA values (other than the outcome). It is unclear how these NA entries are handled in the training of this model. Because the presence or absence of even a single predictor can determine whether a patient receives a CT scan, we will drop the rows containing NA values. 

The two age groups use different sets of predictors, so we will consider them separately. There is no sense eliminating a patient under 2 y.o. because they are missing information only useful to the $\geq$ 2 y.o. model.

Now that we have the appropriate predictors, we will drop rows that contain NA values. The data documentation tells us that NA entries do not have significant meaning for any of these predictors.

## Age < 2 years

We now split the data into two datasets: one representing patients younger than 2 y.o. and one representing patients older than 2 y.o. In the paper, the two models corresponding to these datasets do not use the exact same set of predictors.

In [250]:
train, tune, test = dset.get_data()
train, tune, test = helper.get_baseline_data(train, tune, test)
#test = pd.concat([tune, test])
train = pd.concat([tune, train])

test = test.loc[:,~test.columns.duplicated()]
train = train.loc[:,~train.columns.duplicated()]
train.head()

,AMS_1,AgeTwoPlus_1,HemaLoc_2_or_3,LocLen_2_3_4,High_impact_InjSev_3,SFxPalp_1_or_2,ActNorm_0,Vomit_1,SFxBas_1,HASeverity_3,outcome
17596,0,0,0,0,0,0,0,0,0,0,0.0
555,0,1,1,1,1,0,1,0,0,0,1.0
7565,1,0,0,0,0,0,1,1,0,0,0.0
41694,0,0,1,0,0,0,0,1,0,0,0.0
22390,0,1,0,1,1,0,0,0,0,0,0.0


In [251]:
train_young = train[train['AgeTwoPlus_1'] == 1]
train_young = train_young[young_predrs]
test_young = test[test['AgeTwoPlus_1'] == 0]
test_young = test_young[young_predrs]

X_train_young = train_young.drop(columns = 'outcome')
y_train_young = train_young['outcome'].to_numpy()

X_test_young = test_young.drop(columns = 'outcome')
y_test_young = test_young['outcome'].to_numpy()


feat_names_young = X_train_young.columns.values

print(X_train_young.shape)
print(X_test_young.shape)
train_young.head()

(8246, 6)
(6151, 6)


,AMS_1,HemaLoc_2_or_3,LocLen_2_3_4,High_impact_InjSev_3,SFxPalp_1_or_2,ActNorm_0,outcome
555,0,1,1,1,0,1,1.0
22390,0,0,1,1,0,0,0.0
7761,0,1,1,0,0,0,0.0
13425,0,0,1,1,0,0,0.0
20967,0,0,0,0,0,0,0.0


In [252]:
mod_young = GreedyRuleListClassifier(max_depth=7)
mod_young.fit(X_train_young, y = y_train_young, feature_names= feat_names_young)

[{'col': 'SFxPalp_1_or_2',
  'index_col': 4,
  'cutoff': 1,
  'val': 0.01152073732718894,
  'flip': False,
  'val_right': 0.10652920962199312,
  'num_pts': 8246,
  'num_pts_right': 291},
 {'col': 'feat 0',
  'index_col': 0,
  'cutoff': 1,
  'val': 0.008045254556882464,
  'flip': False,
  'val_right': 0.03625730994152047,
  'num_pts': 7955,
  'num_pts_right': 855},
 {'col': 'feat 2',
  'index_col': 2,
  'cutoff': 1,
  'val': 0.004647887323943662,
  'flip': False,
  'val_right': 0.03211009174311927,
  'num_pts': 7100,
  'num_pts_right': 218},
 {'col': 'feat 1',
  'index_col': 1,
  'cutoff': 1,
  'val': 0.003777971519907004,
  'flip': False,
  'val_right': 0.014155712841253791,
  'num_pts': 6882,
  'num_pts_right': 989},
 {'col': 'feat 3',
  'index_col': 3,
  'cutoff': 1,
  'val': 0.002036314271169184,
  'flip': False,
  'val_right': 0.005084745762711864,
  'num_pts': 5893,
  'num_pts_right': 1180},
 {'col': 'feat 5',
  'index_col': 5,
  'cutoff': 1,
  'val': 0.001273074474856779,
  'flip

In [253]:
mod_young.print_list()
print(mod_young)

	                                    => 1.15% risk (8246 pts)
                          if SFxPalp_1_or_2 ===> 10.7% risk (291 pts)
	                                    => 0.8% risk (7955 pts)
                                  if feat 0 ===> 3.6% risk (855 pts)
	                                    => 0.46% risk (7100 pts)
                                  if feat 2 ===> 3.2% risk (218 pts)
	                                    => 0.38% risk (6882 pts)
                                  if feat 1 ===> 1.4% risk (989 pts)
	                                    => 0.2% risk (5893 pts)
                                  if feat 3 ===> 0.5% risk (1180 pts)
	                                    => 0.13% risk (4713 pts)
                                  if feat 5 ===> 0.6% risk (348 pts)
	                                    => 0.09% risk (4365 pts)
                                  if feat 5 ===> 0.1% risk (4365 pts)

mean 0.012 (8246 pts)
if SFxPalp_1_or_2 >= 1 then 0.107 (291 pts)
mean 0.008 (795

There is an error in this function from the imodels package. The predictor names never work after the first split, not even in Chandan's example code. The following functions fix the problem.

In [254]:
def fix_rule_names(rule_list_mod, feature_names):
    colname_dict = {}
    keys = ['feat 0', 'feat 1', 'feat 2', 'feat 3', 'feat 4', 'feat 5']
    values = feature_names.tolist()
    for i in range(len(keys)):
        colname_dict[keys[i]] = values[i]
        
    for i in range(len(rule_list_mod.rules_)):
        if list(rule_list_mod.rules_[i].keys())[0] == 'col':
            x = rule_list_mod.rules_[i]['col']
            if x[0:4] == 'feat':
                rule_list_mod.rules_[i]['col'] = colname_dict[rule_list_mod.rules_[i]['col']]
            
    return rule_list_mod

def extract_rules(rule_list_mod):
    rule_names = []
    for i in range(len(rule_list_mod.rules_)):
        if list(rule_list_mod.rules_[i].keys())[0] == 'col':
            rule_names.append(rule_list_mod.rules_[i]['col'])

    return rule_names

In [255]:
mod_young_corrected = fix_rule_names(mod_young, feat_names_young)
mod_young_corrected.print_list()
print(mod_young_corrected)

	                                    => 1.15% risk (8246 pts)
                          if SFxPalp_1_or_2 ===> 10.7% risk (291 pts)
	                                    => 0.8% risk (7955 pts)
                                   if AMS_1 ===> 3.6% risk (855 pts)
	                                    => 0.46% risk (7100 pts)
                            if LocLen_2_3_4 ===> 3.2% risk (218 pts)
	                                    => 0.38% risk (6882 pts)
                          if HemaLoc_2_or_3 ===> 1.4% risk (989 pts)
	                                    => 0.2% risk (5893 pts)
                    if High_impact_InjSev_3 ===> 0.5% risk (1180 pts)
	                                    => 0.13% risk (4713 pts)
                               if ActNorm_0 ===> 0.6% risk (348 pts)
	                                    => 0.09% risk (4365 pts)
                               if ActNorm_0 ===> 0.1% risk (4365 pts)

mean 0.012 (8246 pts)
if SFxPalp_1_or_2 >= 1 then 0.107 (291 pts)
mean 0.008 (795

In [256]:
extract_rules(mod_young_corrected)

['SFxPalp_1_or_2',
 'AMS_1',
 'LocLen_2_3_4',
 'HemaLoc_2_or_3',
 'High_impact_InjSev_3',
 'ActNorm_0',
 'ActNorm_0']

In [269]:
young_baseline = Baseline()
young_pred = young_baseline.predict(X_test_young)

TN = ((young_pred == 0) & (y_test_young == 0)).sum()
TP = ((young_pred == 1) & (y_test_young == 1)).sum()
FN = ((young_pred == 0) & (y_test_young == 1)).sum()
FP = ((young_pred == 1) & (y_test_young == 0)).sum()


print("Sensitivity: ", TP/(TP + FN))
print("NPV: ", TN/(TN + FN))
print("Specificity: ", TN/(TN + FP))
print("Accuracy: ", (young_pred == y_test_young).sum()/young_pred.shape[0])

Sensitivity:  0.868421052631579
NPV:  0.9976370510396976
Specificity:  0.6949794238683128
Accuracy:  0.6971224191188424


In [272]:
young_baseline.print_model(X_test_young)

   Remaining patients have  0.88 % chance of ciTBI
if  SFxPalp_1_or_2 == 1  then  9.2 % chance of ciTBI
   Remaining patients have  0.58 % chance of ciTBI
if  AMS_1 == 1  then  3.1 % chance of ciTBI
   Remaining patients have  0.28 % chance of ciTBI
if  High_impact_InjSev_3 == 1  then  0.9 % chance of ciTBI
   Remaining patients have  0.11 % chance of ciTBI
if  LocLen_2_3_4 == 1  then  1.3 % chance of ciTBI
   Remaining patients have  0.08 % chance of ciTBI
if  HemaLoc_2_or_3 == 1  then  0.3 % chance of ciTBI
   Remaining patients have  0.04 % chance of ciTBI
if  ActNorm_0 == 1  then  0.3 % chance of ciTBI
   Remaining patients have  0.02 % chance of ciTBI


'test'

# Discussion

*Maybe some tables can help in this section*

The paper presents very high sensitivity and negative predictive values. Among children in the validation groups who received CT scans, 24.1% under 2 y.o. and 20.1% over 2 y.o. expressed none of the six predictors in their respective models. The authors claim that the models provide evidence against CT scans for children in those groups. If true, it would seem that doctors using these models would reduce the number of unnecessary CT scans by around 20%.

However, if a patient expresses any one of the predictors, the probability of them having a ciTBI goes up to at least 0.5%. As a result, almost half of the patients in the validation group were assigned probabilities over 0.5%. According to Dr. Nathan Liu, he and many other doctors would order a CT scan if they thought the probability of a ciTBI was any higher than 0.5%. Considering only children under 2 y.o., this model could lead to 1040/2216 (46.93%) being assigned CT scans instead of the original 694/2216 (31.32%). This is almost a 50% increase in the number of CT scans being given to children. 

The paper states that they are concerned with the number of unneccesary CT scans being assigned to children, but their proposed models have the potential to exacerbate the issue. Their focus is solely on maximizing the negative predictive value and sensitivity, which both perform better when there are fewer false negatives. As a result, thier models are overly concerned with not missing a single ciTBI, which is the same reason doctors assign CT scans more often then necessary.

As a result, metrics like accuracy and specificity must also be considered.



## NA values

They somehow included entries that had NA values for at least one of the 6 predictors. We will exclude those because even a single predictor being true is enough to single a ciTBI. *I should ensure that the rate of ciTBIs isn't different among those with and those without NA values.* **Oops** *The distributions aren't the same. Honestly, just leave it at the two sentences you have above.*

## Age >= 2 years

In [258]:
train_old = train[train['AgeTwoPlus_1'] == 0]
train_old = train_old[old_predrs]
test_old = test[test['AgeTwoPlus_1'] == 0]
test_old = test_old[old_predrs]

X_train_old = train_old.drop(columns = 'outcome')
y_train_old = train_old['outcome'].to_numpy()

X_test_old = test_old.drop(columns = 'outcome')
y_test_old = test_old['outcome'].to_numpy()


feat_names_old = X_train_old.columns.values

print(X_train_old.shape)
print(X_test_old.shape)
train_old.head()

(24662, 6)
(6151, 6)


,AMS_1,LocLen_2_3_4,High_impact_InjSev_3,Vomit_1,SFxBas_1,HASeverity_3,outcome
17596,0,0,0,0,0,0,0.0
7565,1,0,0,1,0,0,0.0
41694,0,0,0,1,0,0,0.0
6727,0,0,1,0,0,0,0.0
20846,0,0,0,0,0,0,0.0


In [275]:
mod_old = GreedyRuleListClassifier(max_depth=6)
mod_old.fit(X_train_old, y = y_train_old, feature_names= feat_names_old)

[{'col': 'AMS_1',
  'index_col': 0,
  'cutoff': 1,
  'val': 0.010866920768794096,
  'flip': False,
  'val_right': 0.0511501210653753,
  'num_pts': 24662,
  'num_pts_right': 3304},
 {'col': 'feat 4',
  'index_col': 4,
  'cutoff': 1,
  'val': 0.004635265474295346,
  'flip': False,
  'val_right': 0.12195121951219512,
  'num_pts': 21358,
  'num_pts_right': 123},
 {'col': 'feat 2',
  'index_col': 2,
  'cutoff': 1,
  'val': 0.003955733458912173,
  'flip': False,
  'val_right': 0.012643224022125641,
  'num_pts': 21235,
  'num_pts_right': 2531},
 {'col': 'feat 3',
  'index_col': 3,
  'cutoff': 1,
  'val': 0.002780153977758768,
  'flip': False,
  'val_right': 0.009538152610441768,
  'num_pts': 18704,
  'num_pts_right': 1992},
 {'col': 'feat 5',
  'index_col': 5,
  'cutoff': 1,
  'val': 0.001974629009095261,
  'flip': False,
  'val_right': 0.010869565217391304,
  'num_pts': 16712,
  'num_pts_right': 276},
 {'col': 'feat 1',
  'index_col': 1,
  'cutoff': 1,
  'val': 0.0018252616208323193,
  'flip

In [276]:
mod_old_corrected = fix_rule_names(mod_old, feat_names_old)
mod_old_corrected.print_list()
print(mod_old_corrected)

	                                    => 1.09% risk (24662 pts)
                                   if AMS_1 ===> 5.1% risk (3304 pts)
	                                    => 0.46% risk (21358 pts)
                                if SFxBas_1 ===> 12.2% risk (123 pts)
	                                    => 0.4% risk (21235 pts)
                    if High_impact_InjSev_3 ===> 1.3% risk (2531 pts)
	                                    => 0.28% risk (18704 pts)
                                 if Vomit_1 ===> 1.0% risk (1992 pts)
	                                    => 0.2% risk (16712 pts)
                            if HASeverity_3 ===> 1.1% risk (276 pts)
	                                    => 0.18% risk (16436 pts)
                            if LocLen_2_3_4 ===> 0.3% risk (1196 pts)

mean 0.011 (24662 pts)
if AMS_1 >= 1 then 0.051 (3304 pts)
mean 0.005 (21358 pts)
if SFxBas_1 >= 1 then 0.122 (123 pts)
mean 0.004 (21235 pts)
if High_impact_InjSev_3 >= 1 then 0.013 (2531 pts)
mean 0.003

In [266]:
old_pred_prob = pd.DataFrame(mod_old_corrected.predict_proba(X_test_old))[1]
old_pred = old_pred_prob < 0.005

TN_old = ((old_pred == 0) & (y_test_old == 0)).sum()
TP_old = ((old_pred == 1) & (y_test_old == 1)).sum()
FN_old = ((old_pred == 0) & (y_test_old == 1)).sum()
FP_old = ((old_pred == 1) & (y_test_old == 0)).sum()


print("Sensitivity: ", TP_old/(TP_old + FN_old))
print("NPV: ", TN_old/(TN_old + FN_old))
print("Specificity: ", TN_old/(TN_old + FP_old))
print("Accuracy: ", (old_pred == y_test_old).sum()/old_pred.shape[0])

Sensitivity:  0.039473684210526314
NPV:  0.9643380556912555
Specificity:  0.32493827160493827
Accuracy:  0.3214111526581044


In [267]:
old_pred_prob = pd.DataFrame(mod_old_corrected.predict_proba(X_train_old))[1]
old_pred = old_pred_prob < 0.005

TN_old = ((old_pred == 0) & (y_train_old == 0)).sum()
TP_old = ((old_pred == 1) & (y_train_old == 1)).sum()
FN_old = ((old_pred == 0) & (y_train_old == 1)).sum()
FP_old = ((old_pred == 1) & (y_train_old == 0)).sum()


print("Sensitivity: ", TP_old/(TP_old + FN_old))
print("NPV: ", TN_old/(TN_old + FN_old))
print("Specificity: ", TN_old/(TN_old + FP_old))
print("Accuracy: ", (old_pred == y_train_old).sum()/old_pred.shape[0])

Sensitivity:  0.11194029850746269
NPV:  0.9710673474349624
Specificity:  0.32745757153398375
Accuracy:  0.325115562403698
